# 4강. Spark SQL & Word Count

In [1]:
#import findspark; findspark.init()
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("SimpleTest").getOrCreate()
df = spark.sql('''select 'spark' as hello ''')
df.show()
#spark.stop()

+-----+
|hello|
+-----+
|spark|
+-----+



### word count를 작성하여 보자
1. text파일을 rdd로 변환
1. map을 이용하여 word 기준으로 분리
1. flatMap을 이용하여 한 단어기준으로 row 추출
1. 각 word에 1이라는 count를 value에 부여
1. reduce를 실행하여 각 word 별 count를 집계

In [3]:
#map 부분을 완성해보자
rdd = spark.sparkContext.textFile('data/wiki_wordcount.txt')
rdd1 = rdd.map( ? )
rdd2 = rdd1.flatMap( ? )
rdd3 = rdd2.map( ? )
rdd3.collect()[:10]

SyntaxError: invalid syntax (<ipython-input-3-8973a19c150f>, line 3)

In [ ]:
#reduce 부분을 완성한 후 10개를 출력
rdd4 = rdd3.reduceByKey( ? )
rdd4.top(10)

In [ ]:
#결과를 저장
rdd4.saveAsTextFile('data/word_count_result')

##### 1) structField와 structType을 이용하여 schema를 부여하고 Dataframe을 만들어 보자

<div class="alert alert-warning"/>
- spark.createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True)<br>
 + data를 dataFrame으로 생성한다 schema가 정의되어 있을때 verifySchema가 True면 invalid 할 시 에러를 냄<br>
 + pyspark.sql.types 아래의 StructField로 field의 타입 이름, null 값여부를 지정하고<br>
 + 각 field들을 StructType로 묶어 schema를 만듬<br><br>
- df.show(n=20, truncate=True, vertical=False)<br>
 + df의 데이터를 주어진 n개만큼 출력한다 truncate가 True일 경우 column이 너무 많을 경우 자른다.<br>
 + vertical이 True일 경우 column의 출력형태를 변경함<br><br>
- df.printSchema()<br>
 + df의 schema 형태를 출력함<br><br>
- df.describe(*cols)<br>
 + 주어진 cols이 숫자형 컬럼이라면 기본통계를 추출함<br>

In [4]:
r1 = ['Son', 34, 'Korea']
r2 = ['Kim', 24, 'Japan']
r3 = ['Park', 14, 'China']

from pyspark.sql  import types
sf1 = types.StructField('name', types.StringType(), True)
sf2 = types.StructField('age', types.IntegerType(), True)
sf3 = types.StructField('country', types.StringType(), True)

schema = types.StructType([sf1, sf2, sf3])
rows = [r1, r2, r3]

df_first = spark.createDataFrame(rows, schema)
df_first.show()

+----+---+-------+
|name|age|country|
+----+---+-------+
| Son| 34|  Korea|
| Kim| 24|  Japan|
|Park| 14|  China|
+----+---+-------+



In [5]:
df_first.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- country: string (nullable = true)



In [6]:
df_first.describe("age").show()

+-------+----+
|summary| age|
+-------+----+
|  count|   3|
|   mean|24.0|
| stddev|10.0|
|    min|  14|
|    max|  34|
+-------+----+



<div class="alert alert-warning"/>
- rdd와 마찬가지로 take(n),  count(), collect()같은 기능과 처음 record를 볼 수 있는 head(), first() 등이 존재<br>
- df는 pyspark.sql.types.Row로 구성되어져 있음<br>
- row는 field name을 이용하여 각 field에 값에 접근 할 수 있음<br>

In [7]:
row = df_first.head()
row

Row(name='Son', age=34, country='Korea')

In [8]:
type(row)

pyspark.sql.types.Row

In [9]:
row['name']

'Son'

##### 2) word count를 Spark SQL - Dataframe을 이용하여 만들어 보자

In [10]:
rdd = spark.sparkContext.textFile('data/wiki_wordcount.txt')
rdd.take(10)

['',
 'Word count',
 'From Wikipedia, the free encyclopedia',
 'Jump to navigationJump to search',
 'The word count is the number of words in a document or passage of text. Word counting may be needed when a text is required to stay within certain numbers of words. This may particularly be the case in academia, legal proceedings, journalism and advertising. Word count is commonly used by translators to determine the price for the translation job. Word counts may also be used to calculate measures of readability and to measure typing and reading speeds (usually in words per minute). When converting character counts to words, a measure of 5 or 6 characters to a word is generally used for English.[1]',
 '',
 '',
 'Contents',
 '1\tDetails and variations of definition',
 '2\tSoftware']

<div class="alert alert-warning"/>
- rdd.toDF(schema=None, sampleRatio=None)<br>
 + rdd는 toDF를 통하여 dataframe으로 간단히 변경 시킬 수 있음
 + schema나 sampleRatio를 이용하여 변경될 때 데이터 형식, 컬럼 이름, 변경 비율등을 설정 가능<br>
- df.groupBy(*cols)<br>
 + 주어진 컬럼 형태에 따라 groupby를 실행함. df는 reduce가 없기 때문에 groupBy + agg 연산을 결합하는 것이 일반적<br>
- df.agg(*cols)<br>
 + 주어진 groupedData에 agg용 function을 수행함.
 + from pyspark.sql import functions 참고

In [11]:
df = rdd.flatMap(lambda x:x.split()).map(lambda x:(x,1)).toDF(['word', 'cnt'])
df.show()

+--------------+---+
|          word|cnt|
+--------------+---+
|          Word|  1|
|         count|  1|
|          From|  1|
|    Wikipedia,|  1|
|           the|  1|
|          free|  1|
|  encyclopedia|  1|
|          Jump|  1|
|            to|  1|
|navigationJump|  1|
|            to|  1|
|        search|  1|
|           The|  1|
|          word|  1|
|         count|  1|
|            is|  1|
|           the|  1|
|        number|  1|
|            of|  1|
|         words|  1|
+--------------+---+
only showing top 20 rows



In [12]:
df.describe().show()

+-------+-----------------+----+
|summary|             word| cnt|
+-------+-----------------+----+
|  count|             1276|1276|
|   mean|            508.3| 1.0|
| stddev|889.1919513332257| 0.0|
|    min|          "Agents|   1|
|    max|            فارسی|   1|
+-------+-----------------+----+



In [13]:
df.printSchema()

root
 |-- word: string (nullable = true)
 |-- cnt: long (nullable = true)



In [14]:
df.groupBy('word').count().orderBy(["count", "word"], ascending=[0, 1]).show()

+-----+-----+
| word|count|
+-----+-----+
|  the|   50|
|   of|   40|
|    a|   36|
|  and|   27|
|   to|   26|
|   is|   22|
| word|   22|
|   in|   14|
|   on|   14|
|words|   14|
|  for|   12|
|  The|   10|
| Word|   10|
|   as|   10|
|count|    9|
|  may|    9|
|   or|    9|
|   be|    8|
| also|    6|
|   by|    6|
+-----+-----+
only showing top 20 rows



In [15]:
from pyspark.sql import functions as func
from pyspark.sql import Column
df.groupBy('word').agg(func.count(df.cnt).alias('count')).orderBy(["count", "word"], ascending=[0, 1]).show()

+-----+-----+
| word|count|
+-----+-----+
|  the|   50|
|   of|   40|
|    a|   36|
|  and|   27|
|   to|   26|
|   is|   22|
| word|   22|
|   in|   14|
|   on|   14|
|words|   14|
|  for|   12|
|  The|   10|
| Word|   10|
|   as|   10|
|count|    9|
|  may|    9|
|   or|    9|
|   be|    8|
| also|    6|
|   by|    6|
+-----+-----+
only showing top 20 rows



##### 3) DataFrameReader/DataFrameWriter를 이용하여 입출력을 수행하여보자
- 외부 데이터를 df로 읽거나 쓰기 위해서 read/write 메서드를 지원함
- read 순서
 + read 메서드 호출 -> DataFrameReader 인스턴스 생성
 + format 메서드를 이용하여 read되는 데이터 소스 유형을 지정 (text, csv, json, jdbc, kafka,parquet, console, socket 등)
 + option format 별 상세 옵션 지정
 + load 메서드로 실제 df를 생성

In [16]:
df_wordcnt = spark.read.format('text').load('data/wiki_wordcount.txt')
df_wordcnt.show(5)

+--------------------+
|               value|
+--------------------+
|                    |
|          Word count|
|From Wikipedia, t...|
|Jump to navigatio...|
|The word count is...|
+--------------------+
only showing top 5 rows



In [17]:
df_wordcnt = df_wordcnt.withColumnRenamed("value", "text")
df_wordcnt.show(5)

+--------------------+
|                text|
+--------------------+
|                    |
|          Word count|
|From Wikipedia, t...|
|Jump to navigatio...|
|The word count is...|
+--------------------+
only showing top 5 rows



- write 순서
 + write 메서드 호출 -> DataFrameWriter 인스턴스 생성
 + format 메서드를 이용하여 read되는 데이터 소스 유형을 지정 (text, csv, json, jdbc, kafka,parquet, console, socket 등)
 + option format 별 상세 옵션 지정
 + save 메서드로 실제 파일을 생성

In [18]:
df_first.write.format('csv').mode("overwrite").save('data/df_first.csv')
df_first.write.format('json').mode("overwrite").save('data/df_first.json')

<div class="alert alert-warning"/>
- spark.createDataFrame(*cols)<br>
 + 해당 메서드를 이용하여 pandas dataframe을 바로 spark dataframe으로 변경하는 것도 가능

In [19]:
import pandas as pd
df_pd = pd.DataFrame([("foo", 1), ("bar", 2)], columns=("k", "v"))
spark.createDataFrame(df_pd).show()

+---+---+
|  k|  v|
+---+---+
|foo|  1|
|bar|  2|
+---+---+



##### 4) DataFrame의 비타입 연산(untyped operation)을 이용한 word count
- dataFrame의 연산을 일반적으로 비타입연산이라고 부름
- 비타입연산은 각각의 row에 대하여 다시 지정된 col의 형태로 로직이 처리되는 형태를 보임

<div class="alert alert-warning"/>
- df.select(*cols)<br>
 + df에서 지정된 cols를 반환함<br>
 + 비타입연산 func을 이용하여 col의 value를 변환시킬 수 있음<br>
 + col은 문자열이외에도 pyspark.sql.functions.col을 이용하여 반환 될 수 있음<br>
 + pyspark.sql.functions.col은 pyspark.sql.Column을 생성함.<br>
 + 해당 인스턴스는 실제 col에 대한 정보를 변경 할 수 있음<br>
- functions.split(str, sep)<br>
 + 주어진 문자열을 주어진 sep으로 분리하여 배열 형태로 저장
- explode(col)<br>
 + 주어진 컬럼안의 배열을 각 row로 분리하여 저장

In [20]:
df_first.select(['country','age']).show(5)

+-------+---+
|country|age|
+-------+---+
|  Korea| 34|
|  Japan| 24|
|  China| 14|
+-------+---+



In [21]:
from pyspark.sql import Column
df_first.select([func.col('country'), func.col('age')]).show(5)

+-------+---+
|country|age|
+-------+---+
|  Korea| 34|
|  Japan| 24|
|  China| 14|
+-------+---+



In [22]:
from pyspark.sql import Column
df_first.select([func.col('country').alias('cty'), func.col('age')]).show(5)

+-----+---+
|  cty|age|
+-----+---+
|Korea| 34|
|Japan| 24|
|China| 14|
+-----+---+



In [23]:
df = spark.createDataFrame([['a b c d e f']])
df.select(func.split(func.col('_1'), ' ')).show()

+------------------+
|      split(_1,  )|
+------------------+
|[a, b, c, d, e, f]|
+------------------+



In [24]:
df = spark.createDataFrame([['a b c d e f']])
df.select(func.explode(func.split(func.col('_1'), ' '))).show()

+---+
|col|
+---+
|  a|
|  b|
|  c|
|  d|
|  e|
|  f|
+---+



### df_wordcnt 를 이용하여 word count를 Dataframe 명령어로 구현해보자
- 정렬은 count 내림 차순 동일한 count를 가진 경우 word 오름차순으로 한다

In [25]:
df1 = df_wordcnt.select(func.explode(func.split(func.col('text'), ' ')).alias('word'))
df2 = df1.groupBy('word').count().orderBy(['count', 'word'], ascending=[0, 1])
df2.show(5)

+----+-----+
|word|count|
+----+-----+
| the|   50|
|  of|   40|
|   a|   36|
| and|   27|
|  to|   26|
+----+-----+
only showing top 5 rows



##### 5) Spark SQL을 이용하여 dataframe을 sql을 이용하여 컨트롤 해보자
- df는 바로 sql문을 이용하여 처리 될 수 없음
- df.createOrReplaceTempView라는 메서드를 통하여 임시의 테이블로 등록된 후 sql로 처리 될 수 있음

<div class="alert alert-warning"/>
- df.createOrReplaceTempViewe(name)<br>
 + df를 주어진 name의 임시 테이블로 생성<br>
 + 해당 테이블은 해당 메서드를 실행한 스파크 세션이 유지되는 동안만 유효함<br>
- df.createOrReplaceGlobalTempView(name)<br>
 + df를 주어진 name의 글로벌 테이블로 생성<br>
 + 해당 테이블은 전역적인 스파크 세션이 유지되는 동안 유효함<br>
- explain()
 + spark.sql의 실행 계획을 출력함<br>

In [26]:
df2.createOrReplaceTempView('wordcnt')

In [27]:
spark.sql('select * from wordcnt').show()

+-----+-----+
| word|count|
+-----+-----+
|  the|   50|
|   of|   40|
|    a|   36|
|  and|   27|
|   to|   26|
|     |   25|
|   is|   22|
| word|   22|
|   in|   14|
|   on|   14|
|words|   14|
|  for|   12|
|  The|   10|
|   as|   10|
| Word|    9|
|count|    9|
|  may|    9|
|   or|    9|
|   be|    8|
| also|    6|
+-----+-----+
only showing top 20 rows



In [28]:
spark.sql('select * from wordcnt where word == "words"').explain()

== Physical Plan ==
*(4) Sort [count#342L DESC NULLS LAST, word#338 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#342L DESC NULLS LAST, word#338 ASC NULLS FIRST, 200)
   +- *(3) HashAggregate(keys=[word#338], functions=[count(1)])
      +- Exchange hashpartitioning(word#338, 200)
         +- *(2) HashAggregate(keys=[word#338], functions=[partial_count(1)])
            +- *(2) Filter (isnotnull(word#338) && (word#338 = words))
               +- Generate explode(split(text#269,  )), false, [word#338]
                  +- *(1) Project [value#262 AS text#269]
                     +- *(1) FileScan text [value#262] Batched: false, Format: Text, Location: InMemoryFileIndex[file:/lecture/진흥원/data/wiki_wordcount.txt], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<value:string>


In [29]:
spark.sql('select * from wordcnt where word == "words"').show()

+-----+-----+
| word|count|
+-----+-----+
|words|   14|
+-----+-----+



##### 6) udf (user define function) 을 이용하여 custom function을 수행해보자

<div class="alert alert-warning"/>
- pyspark.sql.functions.udf(f=None, returnType=StringType)<br>
 + f에서 수행될 function, returnType에서 pyspark.sql.types의 타입 중 수행후 반환될 타입을 지정히여 udf를 생성할 수 있음<br>
 + 해당 udf는 dataframe에서는 바로 사용 될 수 있지만, spark.sql 에서는 사용될 수 없음
- spark.udf.register(name, f=None, returnType=StringType)<br>
 + 해당 등록 과정을 통하여 spark.sql안에서도 udf를 사용할 수 있음, 지정된 name으로 지정한 f가 수행 됨
- df.withColumn(colName, col)<br>
 + 지정된 colName의 값들을 col에 지정된 형태로 변경함<br>
 + colName이 신규이면 새로운 col을 생성함<br>
- Column.cast(type)<br>
 + 주어진 컬럼의 데이터 타입을 type으로 변경함

In [30]:
array_sum = func.udf(lambda arr: sum(arr), types.IntegerType())

In [31]:
df = spark.createDataFrame([['1,2,3,4,5,6']])
df = df.select(func.split('_1', ',').alias('val'))
df = df.withColumn("val", df.val.cast("array<integer>"))
df.createOrReplaceTempView('test')
df.show()
df.printSchema()

+------------------+
|               val|
+------------------+
|[1, 2, 3, 4, 5, 6]|
+------------------+

root
 |-- val: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [32]:
spark.udf.register('array_sum', array_sum)

<function __main__.<lambda>(arr)>

In [33]:
spark.sql('select *, array_sum(val) from test').show()

+------------------+--------------+
|               val|array_sum(val)|
+------------------+--------------+
|[1, 2, 3, 4, 5, 6]|            21|
+------------------+--------------+



In [34]:
spark.sql('select *, array_sum(val) as sum_val from test').show()

+------------------+-------+
|               val|sum_val|
+------------------+-------+
|[1, 2, 3, 4, 5, 6]|     21|
+------------------+-------+



## 그외 Spark SQL 명령어는 너무나도 많기 때문에 다음 주소를 참조

https://spark.apache.org/docs/latest/sql-programming-guide.html

# test yourself

* word count를 df와 df 펑션을(spark sql 제외) 이용하여 대문자를 소문자로 전환, 특수문자를 제거 하고 집계 하시오 

* udf와 saprk sql을 이용하여 word count를 대문자를 소문자로 전환 후 특수문자를 제거하고 집계하시오